In [5]:
import cv2
import numpy as np


In [3]:
from keras.datasets import cifar100

In [4]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Define the number of bins for the histogram
num_bins = 256

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 1613s 10us/step


In [6]:
# Define HOG parameters
cell_size = (4, 4)           # Size of each cell in pixels
block_size = (2, 2)          # Number of cells in a block
block_stride = (4, 4)        # Step size for block movement
win_size = (32, 32)          # Same as image size
nbins = 9                    # Number of orientation bins

# Initialize HOG descriptor
hog = cv2.HOGDescriptor(
    _winSize=(win_size[1], win_size[0]),
    _blockSize=(block_size[1] * cell_size[1], block_size[0] * cell_size[0]),
    _blockStride=(block_stride[1], block_stride[0]),
    _cellSize=(cell_size[1], cell_size[0]),
    _nbins=nbins
)

# Function to compute HOG features for the dataset
def compute_hog_features(images):
    features = []
    for img in images:
        hog_feats = hog.compute(img).flatten()  # Flatten to 1D vector
        features.append(hog_feats)
    return np.array(features)



In [7]:
# Convert images to grayscale
x_train_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_train])
x_test_gray = np.array([cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in x_test])

# Extract HOG features for training and test datasets
x_train_hog = compute_hog_features(x_train_gray)
x_test_hog = compute_hog_features(x_test_gray)

print(f"Training HOG feature shape: {x_train_hog.shape}")
print(f"Test HOG feature shape: {x_test_hog.shape}")

Training HOG feature shape: (50000, 1764)
Test HOG feature shape: (10000, 1764)


In [8]:
# Function to extract color histogram features
def color_histogram(image):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Calculate histograms for each color channel
    hist_h = cv2.calcHist([hsv_image], [0], None, [num_bins], [0, 180])  # Hue
    hist_s = cv2.calcHist([hsv_image], [1], None, [num_bins], [0, 256])  # Saturation
    hist_v = cv2.calcHist([hsv_image], [2], None, [num_bins], [0, 256])  # Value

    # Normalize histograms
    cv2.normalize(hist_h, hist_h, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_s, hist_s, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist_v, hist_v, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    # Concatenate histograms into a single feature vector
    histogram = np.concatenate((hist_h, hist_s, hist_v), axis=0)

    return histogram

In [9]:
# Extract features from training and test data
train_features_color = []
for image in x_train:
    feature_c = color_histogram(image)
    train_features_color.append(feature_c)

test_features_color = []
for image in x_test:
    feature_c_test = color_histogram(image)
    test_features_color.append(feature_c_test)

# Convert feature lists to NumPy arrays
train_features_color = np.array(train_features_color)
test_features_color = np.array(test_features_color)

print("Training features shape:", train_features_color.shape)
print("Test features shape:", test_features_color.shape)

Training features shape: (50000, 768, 1)
Test features shape: (10000, 768, 1)


In [10]:
# Function to compute LBP histogram
def lbp_histogram(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply LBP operator
    lbp = cv2.calcHist([gray], [0], None, [256], [0, 256])
    lbp = cv2.normalize(lbp, lbp, 0, 255, cv2.NORM_MINMAX)

    # Flatten the histogram
    hist = lbp.flatten()

    return hist

In [25]:
# Extract features from training
train_features_lbp = []
for image in x_train:
    feature_lbp = lbp_histogram(image)
    train_features_lbp.append(feature_lbp)
    
# Extract features from test data
test_features_lbp = []
for image in x_test:
    feature_lbp = lbp_histogram(image)
    test_features_lbp.append(feature_lbp)
    
# Convert feature lists to NumPy arrays
train_features_lbp = np.array(train_features_lbp)
test_features_lbp = np.array(test_features_lbp)

print("Training features shape:", train_features_lbp.shape)
print("Test features shape:", test_features_lbp.shape)

Training features shape: (50000, 256)
Test features shape: (10000, 256)
